In [1]:
import gc
import os
import itertools
import pickle
import re
import time
from random import choice, choices
from functools import reduce
from tqdm import tqdm
from itertools import cycle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from functools import reduce
from itertools import cycle
from scipy import stats
from scipy.stats import skew, kurtosis
from sklearn import metrics, model_selection, preprocessing, linear_model, ensemble, decomposition, tree
import lightgbm as lgb
import optuna
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb
import pandas as pd
import numpy as np
import re
import copy
from typing import List, Tuple, Dict
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import polars as pl


import sys
sys.path.append('../')
from utils import *

In [2]:
is_kaggle = False

if is_kaggle:
    base_dir = '/kaggle/input'
    data_dir = f'{base_dir}/linking-writing-processes-to-writing-quality'
    output_dir = '/kaggle/working'
else:
    base_dir = '../'
    data_dir = f'{base_dir}/data'
    models_dir = f'{base_dir}/models'

In [3]:
# train_logs_df = pd.read_csv(f'../kaggle feat_eng/train_logs_corrected.csv')
# train_scores_df = pd.read_csv(f'{data_dir}/train_scores.csv')

# test_logs_df = pd.read_csv(f'../kaggle feat_eng/test_logs_corrected.csv')

In [4]:
train_logs = pl.scan_csv('../kaggle feat_eng/train_logs_corrected.csv')

# Feats

## Features related to counts and bursts

In [5]:
num_cols = ['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']
activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',', 'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified']
text_changes = ['q', ' ', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':']


def count_by_values(df, colname, values):
    fts = df.select(pl.col('id').unique(maintain_order=True))
    for i, value in enumerate(values):
        tmp_df = df.group_by('id').agg(pl.col(colname).is_in([value]).sum().alias(f'{colname}_{i}_cnt'))
        fts  = fts.join(tmp_df, on='id', how='left') 
    return fts


def dev_feats(df):
    
    print("< Count by values features >")
    
    feats = count_by_values(df, 'activity', activities)
    feats = feats.join(count_by_values(df, 'text_change', text_changes), on='id', how='left') 
    feats = feats.join(count_by_values(df, 'down_event', events), on='id', how='left') 
    feats = feats.join(count_by_values(df, 'up_event', events), on='id', how='left') 

    print("< Input words stats features >")

    temp = df.filter((~pl.col('text_change').str.contains('=>')) & (pl.col('text_change') != 'NoChange'))
    temp = temp.group_by('id').agg(pl.col('text_change').str.concat('').str.extract_all(r'q+'))
    temp = temp.with_columns(input_word_count = pl.col('text_change').list.len(),
                             input_word_length_mean = pl.col('text_change').map_elements(lambda x: np.mean([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_max = pl.col('text_change').map_elements(lambda x: np.max([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_std = pl.col('text_change').map_elements(lambda x: np.std([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_median = pl.col('text_change').map_elements(lambda x: np.median([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_skew = pl.col('text_change').map_elements(lambda x: skew([len(i) for i in x] if len(x) > 0 else 0)))
    temp = temp.drop('text_change')
    feats = feats.join(temp, on='id', how='left') 


    
    print("< Numerical columns features >")

    temp = df.group_by("id").agg(pl.sum('action_time').name.suffix('_sum'), pl.mean(num_cols).name.suffix('_mean'), pl.std(num_cols).name.suffix('_std'),
                                 pl.median(num_cols).name.suffix('_median'), pl.min(num_cols).name.suffix('_min'), pl.max(num_cols).name.suffix('_max'),
                                 pl.quantile(num_cols, 0.5).name.suffix('_quantile'))
    feats = feats.join(temp, on='id', how='left') 


    print("< Categorical columns features >")
    
    temp  = df.group_by("id").agg(pl.n_unique(['activity', 'down_event', 'up_event', 'text_change']))
    feats = feats.join(temp, on='id', how='left') 


    
    print("< Idle time features >")

    temp = df.with_columns(pl.col('up_time').shift().over('id').alias('up_time_lagged'))
    temp = temp.with_columns((abs(pl.col('down_time') - pl.col('up_time_lagged')) / 1000).fill_null(0).alias('time_diff'))
    temp = temp.filter(pl.col('activity').is_in(['Input', 'Remove/Cut']))
    temp = temp.group_by("id").agg(inter_key_largest_lantency = pl.max('time_diff'),
                                   inter_key_median_lantency = pl.median('time_diff'),
                                   mean_pause_time = pl.mean('time_diff'),
                                   std_pause_time = pl.std('time_diff'),
                                   total_pause_time = pl.sum('time_diff'),
                                   pauses_half_sec = pl.col('time_diff').filter((pl.col('time_diff') > 0.5) & (pl.col('time_diff') < 1)).count(),
                                   pauses_1_sec = pl.col('time_diff').filter((pl.col('time_diff') > 1) & (pl.col('time_diff') < 1.5)).count(),
                                   pauses_1_half_sec = pl.col('time_diff').filter((pl.col('time_diff') > 1.5) & (pl.col('time_diff') < 2)).count(),
                                   pauses_2_sec = pl.col('time_diff').filter((pl.col('time_diff') > 2) & (pl.col('time_diff') < 3)).count(),
                                   pauses_3_sec = pl.col('time_diff').filter(pl.col('time_diff') > 3).count(),)
    feats = feats.join(temp, on='id', how='left') 
    
    print("< P-bursts features >")

    temp = df.with_columns(pl.col('up_time').shift().over('id').alias('up_time_lagged'))
    temp = temp.with_columns((abs(pl.col('down_time') - pl.col('up_time_lagged')) / 1000).fill_null(0).alias('time_diff'))
    temp = temp.filter(pl.col('activity').is_in(['Input', 'Remove/Cut']))
    temp = temp.with_columns(pl.col('time_diff')<2)
    temp = temp.with_columns(pl.when(pl.col("time_diff") & pl.col("time_diff").is_last_distinct()).then(pl.count()).over(pl.col("time_diff").rle_id()).alias('P-bursts'))
    temp = temp.drop_nulls()
    temp = temp.group_by("id").agg(pl.mean('P-bursts').name.suffix('_mean'), pl.std('P-bursts').name.suffix('_std'), pl.count('P-bursts').name.suffix('_count'),
                                   pl.median('P-bursts').name.suffix('_median'), pl.max('P-bursts').name.suffix('_max'),
                                   pl.first('P-bursts').name.suffix('_first'), pl.last('P-bursts').name.suffix('_last'))
    feats = feats.join(temp, on='id', how='left') 


    print("< R-bursts features >")

    temp = df.filter(pl.col('activity').is_in(['Input', 'Remove/Cut']))
    temp = temp.with_columns(pl.col('activity').is_in(['Remove/Cut']))
    temp = temp.with_columns(pl.when(pl.col("activity") & pl.col("activity").is_last_distinct()).then(pl.count()).over(pl.col("activity").rle_id()).alias('R-bursts'))
    temp = temp.drop_nulls()
    temp = temp.group_by("id").agg(pl.mean('R-bursts').name.suffix('_mean'), pl.std('R-bursts').name.suffix('_std'), 
                                   pl.median('R-bursts').name.suffix('_median'), pl.max('R-bursts').name.suffix('_max'),
                                   pl.first('R-bursts').name.suffix('_first'), pl.last('R-bursts').name.suffix('_last'))
    feats = feats.join(temp, on='id', how='left')
    
    return feats

In [6]:
train_feats   = dev_feats(train_logs)
train_feats   = train_feats.collect().to_pandas()
for col in train_feats.columns:
    if col != 'id':
        train_feats = train_feats.rename(columns={col: col + '-count_bursts'})
len(train_feats.columns)

< Count by values features >
< Input words stats features >
< Numerical columns features >
< Categorical columns features >
< Idle time features >
< P-bursts features >
< R-bursts features >


116

## Reconstruct essays

In [7]:
def reconstruct_essay(currTextInput):
    essayText = ""
    for Input in currTextInput.values:
        if Input[0] == 'Replace':
            replaceTxt = Input[2].split(' => ')
            essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] + essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
            continue
        if Input[0] == 'Paste':
            essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
            continue
        if Input[0] == 'Remove/Cut':
            essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
            continue
        if "M" in Input[0]:
            croppedTxt = Input[0][10:]
            splitTxt = croppedTxt.split(' To ')
            valueArr = [item.split(', ') for item in splitTxt]
            moveData = (int(valueArr[0][0][1:]), int(valueArr[0][1][:-1]), int(valueArr[1][0][1:]), int(valueArr[1][1][:-1]))
            if moveData[0] != moveData[2]:
                if moveData[0] < moveData[2]:
                    essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
                else:
                    essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
            continue
        essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
    return essayText


def get_essay_df(df):
    df       = df[df.activity != 'Nonproduction']
    temp     = df.groupby('id').apply(lambda x: reconstruct_essay(x[['activity', 'cursor_position', 'text_change']]))
    essay_df = pd.DataFrame({'id': df['id'].unique().tolist()})
    essay_df = essay_df.merge(temp.rename('essay'), on='id')
    return essay_df

In [8]:
train_logs = train_logs.collect().to_pandas()

print('< Essay Reconstruction >')
train_essays = get_essay_df(train_logs)

< Essay Reconstruction >


In [9]:
train_essays.head()

,id,essay
0,001519c8,qqqqqqqqq qq qqqqq qq qqqq qqqq. qqqqqq qqq q...
1,0022f953,"qqqq qq qqqqqqqqqqq ? qq qq qqq qqq qqq, qqqqq..."
2,0042269b,qqqqqqqqqqq qq qqqqq qqqqqqqqq qq qqqqqqqqqqq ...
3,0059420b,qq qqqqqqq qqqqqq qqqqqqqqqqqqq qqqq q qqqq qq...
4,0075873a,"qqqqqqqqqqq qq qqq qqqqq qq qqqqqqqqqq, qqq qq..."


## Aggregations at word/sentence/paragraph/essay level

In [10]:
def q0(x):
    return x.quantile(0.10)
def q1(x):
    return x.quantile(0.25)
def q3(x):
    return x.quantile(0.75)
def q4(x):
    return x.quantile(0.90)

AGGREGATIONS = ['count', 'mean', 'min', 'max', 'first', 'last', q1, 'median', q3, 'sum', q0, q4, 'sem', 'std', 'var', 'skew', pd.DataFrame.kurt]

def word_feats(df):
    df['word'] = df['essay'].apply(lambda x: re.split(' |\\n|\\.|\\?|\\!',x))
    df = df.explode('word')
    df['word_len'] = df['word'].apply(lambda x: len(x))
    df = df[df['word_len'] != 0]

    word_agg_df = df[['id','word_len']].groupby(['id']).agg(AGGREGATIONS)
    word_agg_df.columns = ['_'.join(x) for x in word_agg_df.columns]
    word_agg_df['id'] = word_agg_df.index
    word_agg_df = word_agg_df.reset_index(drop=True)
    return word_agg_df


def sent_feats(df):
    df['sent'] = df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
    df = df.explode('sent')
    df['sent'] = df['sent'].apply(lambda x: x.replace('\n','').strip())
    # Number of characters in sentences
    df['sent_len'] = df['sent'].apply(lambda x: len(x))
    # Number of words in sentences
    df['sent_word_count'] = df['sent'].apply(lambda x: len(x.split(' ')))
    df = df[df.sent_len!=0].reset_index(drop=True)

    sent_agg_df = pd.concat([df[['id','sent_len']].groupby(['id']).agg(AGGREGATIONS), 
                             df[['id','sent_word_count']].groupby(['id']).agg(AGGREGATIONS)], axis=1)
    sent_agg_df.columns = ['_'.join(x) for x in sent_agg_df.columns]
    sent_agg_df['id'] = sent_agg_df.index
    sent_agg_df = sent_agg_df.reset_index(drop=True)
    sent_agg_df.drop(columns=["sent_word_count_count"], inplace=True)
    sent_agg_df = sent_agg_df.rename(columns={"sent_len_count":"sent_count"})
    return sent_agg_df


def parag_feats(df):
    df['paragraph'] = df['essay'].apply(lambda x: x.split('\n'))
    df = df.explode('paragraph')
    # Number of characters in paragraphs
    df['paragraph_len'] = df['paragraph'].apply(lambda x: len(x)) 
    # Number of words in paragraphs
    df['paragraph_word_count'] = df['paragraph'].apply(lambda x: len(x.split(' ')))
    df = df[df.paragraph_len!=0].reset_index(drop=True)
    
    paragraph_agg_df = pd.concat([df[['id','paragraph_len']].groupby(['id']).agg(AGGREGATIONS), 
                                  df[['id','paragraph_word_count']].groupby(['id']).agg(AGGREGATIONS)], axis=1) 
    paragraph_agg_df.columns = ['_'.join(x) for x in paragraph_agg_df.columns]
    paragraph_agg_df['id'] = paragraph_agg_df.index
    paragraph_agg_df = paragraph_agg_df.reset_index(drop=True)
    paragraph_agg_df.drop(columns=["paragraph_word_count_count"], inplace=True)
    paragraph_agg_df = paragraph_agg_df.rename(columns={"paragraph_len_count":"paragraph_count"})
    return paragraph_agg_df

def essay_feats(df):
    # Number of characters in paragraphs
    df['essay_len'] = df['essay'].apply(lambda x: len(x))
    # Number of words in paragraphs
    df['essay_word_count'] = df['essay'].apply(lambda x: len(x.split(' ')))
    return df[['id','essay_len','essay_word_count']]

def product_to_keys(logs, essays):
    essays['product_len'] = essays.essay.str.len()
    tmp_df = logs[logs.activity.isin(['Input', 'Remove/Cut'])].groupby(['id']).agg({'activity': 'count'}).reset_index().rename(columns={'activity': 'keys_pressed'})
    essays = essays.merge(tmp_df, on='id', how='left')
    essays['product_to_keys'] = essays['product_len'] / essays['keys_pressed']
    return essays[['id', 'product_to_keys']]

def get_keys_pressed_per_second(logs):
    temp_df = logs[logs['activity'].isin(['Input', 'Remove/Cut'])].groupby(['id']).agg(keys_pressed=('event_id', 'count')).reset_index()
    temp_df_2 = logs.groupby(['id']).agg(min_down_time=('down_time', 'min'), max_up_time=('up_time', 'max')).reset_index()
    temp_df = temp_df.merge(temp_df_2, on='id', how='left')
    temp_df['keys_per_second'] = temp_df['keys_pressed'] / ((temp_df['max_up_time'] - temp_df['min_down_time']) / 1000)
    return temp_df[['id', 'keys_per_second']]

In [11]:
def append_features(train_feats, temp_feats, suffix):
    for col in temp_feats.columns:
        if col != 'id':
            temp_feats = temp_feats.rename(columns={col: col + suffix})
    train_feats = train_feats.merge(temp_feats, on='id', how='left')
    return train_feats

In [12]:
temp_feats = pd.DataFrame()
temp_feats['id'] = train_essays['id'].unique()
temp_feats = temp_feats.merge(word_feats(train_essays), on='id', how='left')
temp_feats = temp_feats.merge(sent_feats(train_essays), on='id', how='left')
temp_feats = temp_feats.merge(parag_feats(train_essays), on='id', how='left')
temp_feats = temp_feats.merge(essay_feats(train_essays), on='id', how='left')

train_feats = append_features(train_feats, temp_feats, suffix='-word_sent_parag_agg')
print(len(train_feats.columns))

201


In [13]:
temp_feats = pd.DataFrame()
temp_feats['id'] = train_essays['id'].unique()
temp_feats = temp_feats.merge(get_keys_pressed_per_second(train_logs), on='id', how='left')
temp_feats = temp_feats.merge(product_to_keys(train_logs, train_essays), on='id', how='left')

train_feats = append_features(train_feats, temp_feats, suffix='-pressed_keys')
print(len(train_feats.columns))

203


## Count pauses

In [14]:
def count_pauses_2s(group):
    """Counts pauses longer than 2000 ms."""
    gap = group['down_time'] - group['up_time'].shift(1)
    return (gap > 2000).sum()

def count_pauses_5s(group):
    """Counts pauses longer than 2000 ms."""
    gap = group['down_time'] - group['up_time'].shift(1)
    return (gap > 5000).sum()

def pause_proportion_2s(group):
    """Calculates the proportion of pause time to total essay time."""
    gap = group['down_time'] - group['up_time'].shift(1)
    total_pause_time = gap[gap > 2000].sum()
    total_essay_time = group['up_time'].max() - group['down_time'].min()
    return total_pause_time / total_essay_time if total_essay_time else 0

def pause_proportion_5s(group):
    """Calculates the proportion of pause time to total essay time."""
    gap = group['down_time'] - group['up_time'].shift(1)
    total_pause_time = gap[gap > 2000].sum()
    total_essay_time = group['up_time'].max() - group['down_time'].min()
    return total_pause_time / total_essay_time if total_essay_time else 0

def mean_pause_length(group):
    """Calculates the mean length of pauses longer than 2000 ms."""
    gap = group['down_time'] - group['up_time'].shift(1)
    pauses = gap[gap > 2000]
    return pauses.mean() / 1000 if not pauses.empty else 0

def median_pause_length(group):
    """Calculates the mean length of pauses longer than 2000 ms."""
    gap = group['down_time'] - group['up_time'].shift(1)
    pauses = gap[gap > 2000]
    return pauses.median() / 1000 if not pauses.empty else 0

# Method to aggregate all pause-related features
def aggregate_pause_features(df):
    """Aggregates all pause-related features for each essay."""
    grouped = df.groupby('id')
    pause_features = pd.DataFrame()
    pause_features['n_pauses_2s'] = grouped.apply(count_pauses_2s)
    pause_features['n_pauses_5s'] = grouped.apply(count_pauses_5s)
    pause_features['pause_proportion_2s'] = grouped.apply(pause_proportion_2s)
    pause_features['pause_proportion_5s'] = grouped.apply(pause_proportion_5s)
    pause_features['mean_pause_length'] = grouped.apply(mean_pause_length)
    pause_features['median_pause_length'] = grouped.apply(median_pause_length)
    return pause_features

def process_variance(group):
    """Calculates the variance in the writing process over time for each essay."""
    if len(group) < 2:  # Handling for groups with a single row
        return 0

    bins = np.linspace(group['down_time'].min(), group['up_time'].max(), 11)
    divisions = pd.cut(group['down_time'], bins=bins, include_lowest=True, labels=range(1, 11))
    production_deciles = group.groupby(divisions).agg(n_events=('event_id', 'count'))
    return np.std(production_deciles['n_events'], ddof=1)

def aggregate_process_variance(df):
    """Aggregates the process variance feature for each essay."""
    return df.groupby('id').apply(process_variance).rename('process_variance').to_frame()

In [15]:
temp_feats = pd.DataFrame()
temp_feats['id'] = train_essays['id'].unique()
temp_feats = temp_feats.merge(aggregate_pause_features(train_logs).reset_index(), on='id', how='left')
temp_feats = temp_feats.merge(aggregate_process_variance(train_logs).reset_index(), on='id', how='left')

train_feats = append_features(train_feats, temp_feats, suffix='-paussed_features')
print(len(train_feats.columns))

210


## Calculate cursor visits to different segments

In [16]:
'''
Needs polars dataset
'''

# print("< Cursor moving animation >")

# # Adding 'pos' and 'line' columns
# df = df.with_columns([
#     (pl.col('cursor_position') % 30).alias('pos'),
#     (pl.col('cursor_position') / 30).cast(pl.Int32).alias('line')
# ])

# # Adding 'dist_moved' column
# df = df.with_columns(
#     [pl.col('cursor_position').diff().over('id').fill_null(0).abs().alias('dist_moved')]
# )

# # Calculating average distance moved per event
# avg_dist_per_event = df.groupby('id').agg(
#     pl.mean('dist_moved').alias('avg_dist_per_event')
# )

# # Joining with the feature set
# feats = feats.join(avg_dist_per_event, on='id', how='left')

# # Adding 'line_change' column
# df = df.with_columns(
#     [pl.col('line').diff().over('id').fill_null(0).ne(0).alias('line_change')]
# )

# # Calculating revisits per line and first line revisits
# revisits_per_line = df.filter(pl.col('line_change') == False).group_by(['id', 'line']).agg(
#     pl.count().alias('revisit_count')
# )
# first_line_revisits = revisits_per_line.filter(pl.col('line') == 0).group_by('id').agg(
#     pl.sum('revisit_count').alias('first_line_revisits')
# )

# # Joining with the feature set
# feats = feats.join(first_line_revisits, on='id', how='left')

# # Calculating line changes
# line_changes = df.groupby('id').agg(
#     pl.sum('line_change').alias('line_changes')
# )

# # Joining with the feature set
# feats = feats.join(line_changes, on='id', how='left')

'\nNeeds polars dataset\n'

In [17]:
def calculate_segment_visits(train_logs, train_essays):
    # Calculate the end position of the intro and body for each essay
    train_essays['intro_end'] = train_essays['essay'].apply(lambda x: len(x.split('\n')[0]))
    train_essays['body_end'] = train_essays['essay'].apply(lambda x: len('\n'.join(x.split('\n')[:-1])))

    # Create a dictionary for quick lookup
    ends_dict = train_essays.set_index('id')[['intro_end', 'body_end']].to_dict('index')

    # Function to categorize position
    def categorize_position(row):
        intro_end = ends_dict[row['id']]['intro_end']
        body_end = ends_dict[row['id']]['body_end']
        if row['cursor_position'] <= intro_end:
            return 'intro'
        elif row['cursor_position'] <= body_end:
            return 'body'
        else:
            return 'conclusion'

    # Categorize cursor positions
    train_logs['segment'] = train_logs.apply(categorize_position, axis=1)

    # Count visits to each segment
    segment_visits = train_logs.groupby(['id', 'segment']).size().unstack(fill_value=0)

    # Rename columns for clarity
    segment_visits.columns = [f'{col}_visits' for col in segment_visits.columns]

    return segment_visits

In [18]:
temp_feats = pd.DataFrame()
temp_feats['id'] = train_essays['id'].unique()
temp_feats = temp_feats.merge(calculate_segment_visits(train_logs, train_essays).reset_index(), on='id', how='left')

train_feats = append_features(train_feats, temp_feats, suffix='-segments_visit')
print(len(train_feats.columns))

213


## Relative size of paragraphs

In [19]:
def calculate_relative_paragraph_sizes(input_df, essay_column):

    df = input_df.copy()
    # Split the essay text into paragraphs
    df['paragraphs'] = df[essay_column].str.split('\n')

    # Filter out empty paragraphs
    df['paragraphs'] = df['paragraphs'].apply(lambda paragraphs: [p for p in paragraphs if p.strip() != ''])

    # Calculate the total number of paragraphs
    df['total_paragraphs'] = df['paragraphs'].apply(len)

    # Calculate the relative sizes
    df['relative_intro_size'] = 1 / df['total_paragraphs']  # First paragraph is the introduction
    df['relative_body_size'] = (df['total_paragraphs'] - 2) / df['total_paragraphs']  # Middle paragraphs are the body
    df['relative_conclusion_size'] = 1 / df['total_paragraphs']  # Last paragraph is the conclusion

    # Calculate the word count for each paragraph
    df['paragraph_word_count'] = df['paragraphs'].apply(lambda x: [len(paragraph.split()) for paragraph in x])

    # Separate paragraphs into intro, body, and conclusion
    df['word_count_intro'] = df['paragraph_word_count'].apply(lambda x: x[0] if len(x) > 0 else 0)
    df['word_count_body'] = df['paragraph_word_count'].apply(lambda x: sum(x[1:-1]) if len(x) > 2 else 0)
    df['word_count_conclusion'] = df['paragraph_word_count'].apply(lambda x: x[-1] if len(x) > 1 else 0)

    # Calculate total word count for each essay
    df['total_word_count'] = df['paragraph_word_count'].apply(sum)
    
    # Calculate ratios
    df['intro_ratio'] = df['word_count_intro'] / df['total_word_count']
    df['body_ratio'] = df['word_count_body'] / df['total_word_count']
    df['conclusion_ratio'] = df['word_count_conclusion'] / df['total_word_count']
    
    df['intro_body_ratio'] = df['word_count_intro'] / df['word_count_body']
    df['intro_conclusion_ratio'] = df['word_count_intro'] / df['word_count_conclusion']
    df['body_conclusion_ratio'] = df['word_count_body'] / df['word_count_conclusion']

    # Drop intermediate columns if needed
    df.drop(columns=['word', 'sent', 'paragraph', 'paragraphs', 'total_paragraphs', essay_column, 'paragraph_word_count'], inplace=True)

    return df

In [20]:
temp_feats = pd.DataFrame()
temp_feats['id'] = train_essays['id'].unique()
temp_feats = temp_feats.merge(calculate_relative_paragraph_sizes(train_essays, 'essay').reset_index(), on='id', how='left')

train_feats = append_features(train_feats, temp_feats, suffix='-paragraph_ratios')
print(len(train_feats.columns))

232


## TF-IDF

In [21]:
# # Customer tokenizer
# def custom_tokenizer(text):
#     words = re.findall(r'\b\w+\b', text.lower())
#     filtered_words = [word for word in words if word.count('q') < 11] # Do not consider words with more than 10 q's
#     return filtered_words

# tfidf_vect = TfidfVectorizer(tokenizer=custom_tokenizer, ngram_range=(1,2))

# def make_text_features(self, df, column='text_change', fit_transform=True):
#     """Extracts text features using CountVectorizer and TfidfVectorizer, along with custom features."""
#     # Filter and concatenate text changes
#     filtered_df = df[(~df[column].str.contains('=>')) & (df[column] != 'NoChange')]
#     concatenated_texts = filtered_df.groupby('id')[column].apply(' '.join).reset_index()

#     # Apply CountVectorizer and TfidfVectorizer
#     if fit_transform:
#         bow_features = self.count_vect.fit_transform(concatenated_texts[column])
#         tfidf_features = self.tfidf_vect.fit_transform(concatenated_texts[column])
#     else:
#         bow_features = self.count_vect.transform(concatenated_texts[column])
#         tfidf_features = self.tfidf_vect.transform(concatenated_texts[column])

#     # Convert to DataFrame
#     bow_df = pd.DataFrame(bow_features.toarray(), columns=[f'bow_{name}' for name in self.count_vect.get_feature_names_out()])
#     tfidf_df = pd.DataFrame(tfidf_features.toarray(), columns=[f'tfidf_{name}' for name in self.tfidf_vect.get_feature_names_out()])

#     # Custom Feature: Length of each essay
#     custom_features_df = pd.DataFrame({'custom_length': concatenated_texts[column].apply(len)})

#     # Merge all features
#     merged_features = pd.concat([concatenated_texts[['id']], bow_df, tfidf_df, custom_features_df], axis=1)
#     return merged_features

## Gaps

In [22]:
# Define the gap list for time gap features
gap_list = [1, 2, 3, 5, 10, 20, 50, 100]

def compute_time_gaps(df, gap_list):
    """Computes time gaps between events for a list of specified gaps."""
    for gap in gap_list:
        df[f'up_time_shift{gap}'] = df.groupby('id')['up_time'].shift(gap)
        df[f'action_time_gap{gap}'] = df['down_time'] - df[f'up_time_shift{gap}']

    time_gap_cols = [f'action_time_gap{gap}' for gap in gap_list]
    return df[['id'] + time_gap_cols].groupby('id').agg(['mean', 'std', 'min', 'max'])


def compute_cursor_position_change_features(df, gap_list):
    """Computes cursor position change features for specified gaps per 'id'."""
    result = pd.DataFrame()  # Create an empty DataFrame to store the results
    for gap in gap_list:
        col_shift = f'cursor_position_shift{gap}'
        df[col_shift] = df.groupby('id')['cursor_position'].shift(gap)
        df[f'cursor_position_change{gap}'] = df['cursor_position'] - df[col_shift]
        df[f'cursor_position_abs_change{gap}'] = abs(df[f'cursor_position_change{gap}'])
        # Aggregate the results per 'id'
        id_features = df.groupby('id').agg({
            f'cursor_position_change{gap}': 'mean',  # You can choose different aggregation functions as needed
            f'cursor_position_abs_change{gap}': 'mean'
        }).reset_index()
        result = pd.concat([result, id_features], axis=1)  # Concatenate the results

    result = result.loc[:, ~result.columns.duplicated()]  # Remove duplicate columns
    return result


def compute_word_count_change_features(df, gap_list):
    """Computes word count change features for specified gaps per 'id'."""
    result = pd.DataFrame()  # Create an empty DataFrame to store the results
    for gap in gap_list:
        col_shift = f'word_count_shift{gap}'
        df[col_shift] = df.groupby('id')['word_count'].shift(gap)
        df[f'word_count_change{gap}'] = df['word_count'] - df[col_shift]
        df[f'word_count_abs_change{gap}'] = abs(df[f'word_count_change{gap}'])
        # Aggregate the results per 'id'
        id_features = df.groupby('id').agg({
            f'word_count_change{gap}': 'mean',  # You can choose different aggregation functions as needed
            f'word_count_abs_change{gap}': 'mean'
        }).reset_index()
        result = pd.concat([result, id_features], axis=1)  # Concatenate the results

    result = result.loc[:, ~result.columns.duplicated()]  # Remove duplicate columns
    return result

In [23]:
# Compute time gap features
time_gap_features = compute_time_gaps(train_logs, gap_list)
time_gap_features.columns = ['{}_{}'.format(col1, col2) for col1, col2 in time_gap_features.columns]
time_gap_features = time_gap_features.reset_index()

train_feats = append_features(train_feats, time_gap_features, suffix='-time_gaps')
print(len(train_feats.columns))

264


In [24]:
temp_feats = pd.DataFrame()
temp_feats['id'] = train_essays['id'].unique()
temp_feats = temp_feats.merge(compute_cursor_position_change_features(train_logs, gap_list))
temp_feats = temp_feats.merge(compute_word_count_change_features(train_logs, gap_list))

train_feats = append_features(train_feats, temp_feats, suffix='-cursor_word_changes')
print(len(train_feats.columns))

296


# Punctuation counts

In [25]:
def match_punctuations(df):
    """Counts the number of punctuation marks used in each essay."""
    punctuations = ['"', '.', ',', "'", '-', ';', ':', '?', '!', '<', '>', '/',
                    '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+']

    # Filter the DataFrame to include only rows with punctuation events
    punctuation_df = df[df['down_event'].isin(punctuations)]
    punctuation_df = df[~df['down_event'].isin(['"','#','$','%',')','*',';','<','?','@','^'])] # Not useful characters

    # Group by 'id' and 'down_event' and count the occurrences of each punctuation
    punctuation_counts = punctuation_df.groupby(['id', 'down_event'])['down_event'].count().unstack(fill_value=0)

    # Calculate the total punctuation count for each 'id'
    total_punctuation_counts = punctuation_counts.sum(axis=1)

    # Add the total count as a new column
    punctuation_counts['Total'] = total_punctuation_counts

    return punctuation_counts

In [26]:
temp_feats = pd.DataFrame()
temp_feats['id'] = train_essays['id'].unique()
temp_feats = temp_feats.merge(match_punctuations(train_logs).reset_index(), on='id', how='left')

train_feats = append_features(train_feats, temp_feats, suffix='-punctuation')
print(len(train_feats.columns))

417


## Get input words?

In [27]:
# def get_input_words(df: pd.DataFrame) -> pd.DataFrame:
#     """Extracts and aggregates information about input words from the text changes in the dataset."""
#     # Filter relevant rows and reset index
#     filtered_df = df[(~df['text_change'].str.contains('=>')) & (df['text_change'] != 'NoChange')].reset_index(drop=True)

#     # Group and concatenate text changes
#     grouped_df = filtered_df.groupby('id')['text_change'].apply(''.join).reset_index()

#     # Find all occurrences of 'q+'
#     grouped_df['input_words'] = grouped_df['text_change'].apply(lambda x: re.findall(r'q+', x))

#     # Calculate various statistics
#     stats_df = grouped_df['input_words'].apply(lambda words: pd.Series({
#         'input_word_count': len(words),
#         'input_word_length_mean': np.mean([len(word) for word in words]) if words else 0,
#         'input_word_length_max': np.max([len(word) for word in words]) if words else 0,
#         'input_word_length_std': np.std([len(word) for word in words]) if words else 0
#     }))

#     return pd.concat([grouped_df[['id']], stats_df], axis=1)

## Keyboard / mouse feats

In [28]:
def get_keyboard_mouse_feats(train_logs_df):

    # Creates two shift variables which lag the original variable by 1 and 2 periods respectively.
    event_df = train_logs_df[['id', 'event_id', 'down_event']].copy(deep=True)

    event_df['down_event_shift_1'] = event_df['down_event'].shift(periods=1)

    event_df = event_df[['id', 'event_id', 'down_event_shift_1', 'down_event']]

    ctrl_x_df = ((event_df['down_event_shift_1'] == 'Control') & (event_df['down_event'].str.lower() == 'x')).groupby(event_df['id']).sum().reset_index(name='count')

    # Creating a DataFrame that contains all counts at an id level.
    kb_shortcut_df = pd.DataFrame(event_df['id'].unique(), columns=['id'])
    kb_shortcut_df['ctrl_x_cnt'] = ctrl_x_df['count']

    # Calculating the proportion of mouse click events
    mouse_event_df = pd.DataFrame(train_logs_df['id'].unique(), columns=['id'])
    
    mouse_event_df['mouse_event_cnt'] = train_logs_df.groupby(train_logs_df['id'])['down_event'].apply(lambda x: (x.isin(['Leftclick', 'Rightclick', 'Middleclick', 'Unknownclick']).sum())).reset_index()['down_event']

    mouse_event_df['all_event_cnt'] = train_logs_df.groupby(train_logs_df['id'])['event_id'].max().reset_index()['event_id']

    mouse_event_df['mouse_event_perc'] = (mouse_event_df['mouse_event_cnt']/mouse_event_df['all_event_cnt'])*100.0
    
    return kb_shortcut_df.merge(mouse_event_df, on='id')

In [29]:
temp_feats = pd.DataFrame()
temp_feats['id'] = train_essays['id'].unique()
temp_feats = temp_feats.merge(get_keyboard_mouse_feats(train_logs).reset_index(), on='id', how='left')

train_feats = append_features(train_feats, temp_feats, suffix='-key_mouse')
print(len(train_feats.columns))

422


## Time features

In [30]:
def create_time_features(df):
    """Generates aggregated time-related features for each essay ID."""
    df = df.copy()
    # Time-based calculations
    df['action_time_sec'] = (df['up_time'] - df['down_time']) / 1000.0
    df['time_since_last_event'] = df.groupby('id')['down_time'].diff() / 1000.0
    df['cumulative_action_time'] = df.groupby('id')['action_time_sec'].cumsum()

    # Prepare aggregation dictionary
    aggregations = {
        'action_time_sec': ['mean', 'sum', 'max', 'std'],
        'time_since_last_event': ['mean', 'max', 'std'],
        'cumulative_action_time': ['max']
    }

    # Add rolling window features to aggregations
    for window in [5, 10, 15, 20, 30, 50]:
        df[f'rolling_mean_{window}'] = df.groupby('id')['action_time_sec'].transform(lambda x: x.rolling(window).mean())
        df[f'rolling_std_{window}'] = df.groupby('id')['action_time_sec'].transform(lambda x: x.rolling(window).std())
        aggregations[f'rolling_mean_{window}'] = ['mean']
        aggregations[f'rolling_std_{window}'] = ['mean']

    # Aggregating features for each ID
    aggregated_features = df.groupby('id').agg(aggregations)
    aggregated_features.columns = ['_'.join(col) for col in aggregated_features.columns]
    return aggregated_features

def create_additional_time_features(df):
    """Generates additional aggregated time features for each essay ID."""
    df = df.copy()
    df['time_diff'] = abs(df.groupby('id')['down_time'].diff() - df['up_time'].shift(1)) / 1000
    df['time_diff'] = df['time_diff'].fillna(0)  # Handling the first row for each ID

    # Prepare aggregation dictionary
    aggregates = {'time_diff': ['max', 'median']}

    # Adding boolean counts for pauses
    for pause in [0.5, 1, 1.5, 2, 3, 5, 10, 20]:
        df[f'pauses_{pause}_sec'] = df['time_diff'].apply(lambda x: x > pause)
        aggregates[f'pauses_{pause}_sec'] = ['sum']

    # Aggregating features for each ID
    additional_features = df.groupby('id').agg(aggregates)
    additional_features.columns = ['_'.join(col) for col in additional_features.columns]
    return additional_features

In [31]:
temp_feats = pd.DataFrame()
temp_feats['id'] = train_essays['id'].unique()
temp_feats = temp_feats.merge(create_time_features(train_logs).reset_index(), on='id', how='left')
temp_feats = temp_feats.merge(create_additional_time_features(train_logs).reset_index(), on='id', how='left')

train_feats = append_features(train_feats, temp_feats, suffix='-time_feat')
print(len(train_feats.columns))

452


# Remove useless columns

In [32]:
def filter_columns(df):
    # Initialize lists to store column names for each condition
    all_nan_columns = []
    all_inf_columns = []
    single_value_columns = []

    # Iterate through columns
    for col in df.columns:
        # Check if the column contains numeric data
        if pd.api.types.is_numeric_dtype(df[col]):
            # Check for all NaN values
            if df[col].isna().all():
                all_nan_columns.append(col)
            else:
                # Check for all infinite values (positive or negative infinity)
                if np.isinf(df[col]).all():
                    all_inf_columns.append(col)
                else:
                    # Check for more than 90% of columns with only one unique value
                    unique_value_count = df[col].nunique()
                    total_count = df.shape[0]
                    if unique_value_count == 1 and (total_count - df[col].isna().sum()) / total_count > 0.8:
                        single_value_columns.append(col)

    return {
        "AllNaNColumns": all_nan_columns,
        "AllInfColumns": all_inf_columns,
        "SingleValueColumns": single_value_columns
    }

# Example usage:
# Assuming 'df' is your DataFrame
filtered_columns = filter_columns(train_feats)
print("Columns with all NaNs:", filtered_columns["AllNaNColumns"])
print("Columns with all infinite values:", filtered_columns["AllInfColumns"])
print("Columns with more than 90% of the same value:", filtered_columns["SingleValueColumns"])

Columns with all NaNs: []
Columns with all infinite values: []
Columns with more than 90% of the same value: ['cursor_position_min-count_bursts']


In [33]:
train_feats.to_csv('train_452feats.csv', index=False)

In [34]:
import sys
sys.exit()

SystemExit: 

C:\Users\luis.pinto1\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# OLD

In [ ]:
def split_and_compute_lengths(df, column, delimiter, new_column):
    """Split essays in a DataFrame by a delimiter, remove empty items, and compute lengths, word counts, and word count lists of paragraphs/sentences per id."""
    # Split the essays using the specified delimiter
    split_essays = df[column].str.split(delimiter)
    
    # Remove empty items (empty paragraphs/sentences)
    split_essays = split_essays.apply(lambda x: [item.strip() for item in x if item.strip()])
    
    # Compute the lengths of each paragraph/sentence
    lengths = split_essays.apply(lambda x: [len(paragraph) for paragraph in x])
    
    # Compute the word counts of each paragraph/sentence
    word_counts = split_essays.apply(lambda x: [len(paragraph.split()) for paragraph in x])
    
    # Create a new DataFrame to store the results
    result_df = pd.DataFrame({'id': df['id'], new_column+'_len': lengths, new_column+'_word_count': word_counts})
    
    return result_df

def compute_aggregations(df, id_col, agg_columns):
    """Computes specified aggregations for each id in the DataFrame using split_and_compute_lengths."""
    
    # Specify the aggregation functions
    agg_funcs = ['count', 'mean', 'median', 'std', 'min', 'max', 'var', 'sem']
    
    # Group by 'id' and compute the specified aggregations for the specified columns
    df = df.explode(agg_columns)
    agg_df = df.groupby(id_col).agg(agg_funcs)
    agg_df.columns = ['_'.join(col) for col in agg_df.columns]
    
    return agg_df.reset_index()



class FeatureEngineering:

    @staticmethod
    def get_input_words(df: pd.DataFrame) -> pd.DataFrame:
        """Extracts and aggregates information about input words from the text changes in the dataset."""
        # Filter relevant rows and reset index
        filtered_df = df[(~df['text_change'].str.contains('=>')) & (df['text_change'] != 'NoChange')].reset_index(drop=True)
    
        # Group and concatenate text changes
        grouped_df = filtered_df.groupby('id')['text_change'].apply(''.join).reset_index()
    
        # Find all occurrences of 'q+'
        grouped_df['input_words'] = grouped_df['text_change'].apply(lambda x: re.findall(r'q+', x))
    
        # Calculate various statistics
        stats_df = grouped_df['input_words'].apply(lambda words: pd.Series({
            'input_word_count': len(words),
            'input_word_length_mean': np.mean([len(word) for word in words]) if words else 0,
            'input_word_length_max': np.max([len(word) for word in words]) if words else 0,
            'input_word_length_std': np.std([len(word) for word in words]) if words else 0
        }))

        return pd.concat([grouped_df[['id']], stats_df], axis=1)

class FeatureStats:

    @staticmethod
    def get_word_counts(df: pd.DataFrame, id_col: str, word_count_col: str) -> pd.DataFrame:
        """Aggregates the final word count for each essay."""
        return df.groupby(id_col).agg(final_word_count=(word_count_col, 'last'))

# Customer tokenizer
def custom_tokenizer(text):
    words = re.findall(r'\b\w+\b', text.lower())
    filtered_words = [word for word in words if word.count('q') < 11]
    return filtered_words


class Preprocessor:

    def __init__(self):
        self.activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
        self.text_changes = ['q', ' ', 'NoChange', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':']
        self.count_vect = CountVectorizer(tokenizer=custom_tokenizer, ngram_range=(1,2))
        self.tfidf_vect = TfidfVectorizer(tokenizer=custom_tokenizer, ngram_range=(1,2))
        self.idf = {}

    def activity_counts(self, df):
        """Calculates activity counts for each essay."""
        activity_counts = df.groupby('id')['activity'].value_counts().unstack(fill_value=0)
        activity_counts = activity_counts.reindex(columns=self.activities, fill_value=0)

        # Apply IDF scaling if needed
        if not self.idf:
            self.idf = {col: np.log(df.shape[0] / (activity_counts[col].sum() + 1)) for col in self.activities}
        activity_counts = activity_counts.apply(lambda x: (1 + np.log(x)) * self.idf.get(x.name, 0), axis=0)

        return activity_counts.add_prefix('activity_')

    def event_counts(self, df, colname):
        """Calculates event counts for each essay."""
        events = ['ArrowRight', 'ArrowLeft', 'ArrowDown', 'ArrowUp', 'CapsLock', 
                  "'", 'Unidentified']

        event_counts = df.groupby('id')[colname].value_counts().unstack(fill_value=0)
        event_counts = event_counts.reindex(columns=events, fill_value=0)

        # Apply IDF scaling if needed
        if not self.idf:
            self.idf = {col: np.log(df.shape[0] / (event_counts[col].sum() + 1)) for col in events}
        event_counts = event_counts.apply(lambda x: (1 + np.log(x)) * self.idf.get(x.name, 0), axis=0)

        return event_counts.add_prefix(f'{colname}_')


    def text_change_counts(self, df):
        """Calculates counts of different types of text changes for each essay."""
        text_change_counts = df.groupby('id')['text_change'].value_counts().unstack(fill_value=0)
        text_change_counts = text_change_counts.reindex(columns=self.text_changes, fill_value=0)

        return text_change_counts.add_prefix('text_change_')
    
    def match_punctuations(self, df):
        """Counts the number of punctuation marks used in each essay."""
        punctuations = ['"', '.', ',', "'", '-', ';', ':', '?', '!', '<', '>', '/',
                        '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+']

        # Filter the DataFrame to include only rows with punctuation events
        punctuation_df = df[df['down_event'].isin(punctuations)]
        punctuation_df = df[~df['down_event'].isin(['"','#','$','%',')','*',';','<','?','@','^'])]

        # Group by 'id' and 'down_event' and count the occurrences of each punctuation
        punctuation_counts = punctuation_df.groupby(['id', 'down_event'])['down_event'].count().unstack(fill_value=0)

        # Calculate the total punctuation count for each 'id'
        total_punctuation_counts = punctuation_counts.sum(axis=1)

        # Add the total count as a new column
        punctuation_counts['Total'] = total_punctuation_counts

        return punctuation_counts

    def compute_time_gaps(self, df, gap_list):
        """Computes time gaps between events for a list of specified gaps."""
        for gap in gap_list:
            df[f'up_time_shift{gap}'] = df.groupby('id')['up_time'].shift(gap)
            df[f'action_time_gap{gap}'] = df['down_time'] - df[f'up_time_shift{gap}']

        time_gap_cols = [f'action_time_gap{gap}' for gap in gap_list]
        return df[['id'] + time_gap_cols].groupby('id').agg(['mean', 'std', 'min', 'max'])

    @staticmethod
    def count_pauses(group):
        """Counts pauses longer than 2000 ms."""
        gap = group['down_time'] - group['up_time'].shift(1)
        return (gap > 2000).sum()

    @staticmethod
    def pause_proportion(group):
        """Calculates the proportion of pause time to total essay time."""
        gap = group['down_time'] - group['up_time'].shift(1)
        total_pause_time = gap[gap > 2000].sum()
        total_essay_time = group['up_time'].max() - group['down_time'].min()
        return total_pause_time / total_essay_time if total_essay_time else 0

    @staticmethod
    def mean_pause_length(group):
        """Calculates the mean length of pauses longer than 2000 ms."""
        gap = group['down_time'] - group['up_time'].shift(1)
        pauses = gap[gap > 2000]
        return pauses.mean() / 1000 if not pauses.empty else 0

    # Method to aggregate all pause-related features
    def aggregate_pause_features(self, df):
        """Aggregates all pause-related features for each essay."""
        grouped = df.groupby('id')
        pause_features = pd.DataFrame()
        pause_features['n_pauses'] = grouped.apply(self.count_pauses)
        pause_features['pause_proportion'] = grouped.apply(self.pause_proportion)
        pause_features['mean_pause_length'] = grouped.apply(self.mean_pause_length)
        return pause_features

    @staticmethod
    def process_variance(group):
        """Calculates the variance in the writing process over time for each essay."""
        if len(group) < 2:  # Handling for groups with a single row
            return 0

        bins = np.linspace(group['down_time'].min(), group['up_time'].max(), 11)
        divisions = pd.cut(group['down_time'], bins=bins, include_lowest=True, labels=range(1, 11))
        production_deciles = group.groupby(divisions).agg(n_events=('event_id', 'count'))
        return np.std(production_deciles['n_events'], ddof=1)

    def aggregate_process_variance(self, df):
        """Aggregates the process variance feature for each essay."""
        return df.groupby('id').apply(self.process_variance).rename('process_variance').to_frame()

    def create_time_features(self, df):
        """Generates aggregated time-related features for each essay ID."""
        df = df.copy()
        # Time-based calculations
        df['action_time_sec'] = (df['up_time'] - df['down_time']) / 1000.0
        df['time_since_last_event'] = df.groupby('id')['down_time'].diff() / 1000.0
        df['cumulative_action_time'] = df.groupby('id')['action_time_sec'].cumsum()

        # Prepare aggregation dictionary
        aggregations = {
            'action_time_sec': ['mean', 'sum', 'max', 'std'],
            'time_since_last_event': ['mean', 'max', 'std'],
            'cumulative_action_time': ['max']
        }

        # Add rolling window features to aggregations
        for window in [5, 10, 15, 20, 30, 50]:
            df[f'rolling_mean_{window}'] = df.groupby('id')['action_time_sec'].transform(lambda x: x.rolling(window).mean())
            df[f'rolling_std_{window}'] = df.groupby('id')['action_time_sec'].transform(lambda x: x.rolling(window).std())
            aggregations[f'rolling_mean_{window}'] = ['mean']
            aggregations[f'rolling_std_{window}'] = ['mean']

        # Aggregating features for each ID
        aggregated_features = df.groupby('id').agg(aggregations)
        aggregated_features.columns = ['_'.join(col) for col in aggregated_features.columns]
        return aggregated_features.reset_index()

    def create_additional_time_features(self, df):
        """Generates additional aggregated time features for each essay ID."""
        df = df.copy()
        df['time_diff'] = abs(df.groupby('id')['down_time'].diff() - df['up_time'].shift(1)) / 1000
        df['time_diff'] = df['time_diff'].fillna(0)  # Handling the first row for each ID

        # Prepare aggregation dictionary
        aggregates = {'time_diff': ['max', 'median']}

        # Adding boolean counts for pauses
        for pause in [0.5, 1, 1.5, 2, 3, 5, 10, 20]:
            df[f'pauses_{pause}_sec'] = df['time_diff'].apply(lambda x: x > pause)
            aggregates[f'pauses_{pause}_sec'] = ['sum']

        # Aggregating features for each ID
        additional_features = df.groupby('id').agg(aggregates)
        additional_features.columns = ['_'.join(col) for col in additional_features.columns]
        return additional_features.reset_index()


    def make_text_features(self, df, column='text_change', fit_transform=True):
        """Extracts text features using CountVectorizer and TfidfVectorizer, along with custom features."""
        # Filter and concatenate text changes
        filtered_df = df[(~df[column].str.contains('=>')) & (df[column] != 'NoChange')]
        concatenated_texts = filtered_df.groupby('id')[column].apply(' '.join).reset_index()

        # Apply CountVectorizer and TfidfVectorizer
        if fit_transform:
            bow_features = self.count_vect.fit_transform(concatenated_texts[column])
            tfidf_features = self.tfidf_vect.fit_transform(concatenated_texts[column])
        else:
            bow_features = self.count_vect.transform(concatenated_texts[column])
            tfidf_features = self.tfidf_vect.transform(concatenated_texts[column])

        # Convert to DataFrame
        bow_df = pd.DataFrame(bow_features.toarray(), columns=[f'bow_{name}' for name in self.count_vect.get_feature_names_out()])
        tfidf_df = pd.DataFrame(tfidf_features.toarray(), columns=[f'tfidf_{name}' for name in self.tfidf_vect.get_feature_names_out()])

        # Custom Feature: Length of each essay
        custom_features_df = pd.DataFrame({'custom_length': concatenated_texts[column].apply(len)})

        # Merge all features
        merged_features = pd.concat([concatenated_texts[['id']], bow_df, tfidf_df, custom_features_df], axis=1)
        return merged_features

    def compute_cursor_position_change_features(self, df, gap_list):
        """Computes cursor position change features for specified gaps per 'id'."""
        result = pd.DataFrame()  # Create an empty DataFrame to store the results
        for gap in gap_list:
            col_shift = f'cursor_position_shift{gap}'
            df[col_shift] = df.groupby('id')['cursor_position'].shift(gap)
            df[f'cursor_position_change{gap}'] = df['cursor_position'] - df[col_shift]
            df[f'cursor_position_abs_change{gap}'] = abs(df[f'cursor_position_change{gap}'])
            # Aggregate the results per 'id'
            id_features = df.groupby('id').agg({
                f'cursor_position_change{gap}': 'mean',  # You can choose different aggregation functions as needed
                f'cursor_position_abs_change{gap}': 'mean'
            }).reset_index()
            result = pd.concat([result, id_features], axis=1)  # Concatenate the results

        result = result.loc[:, ~result.columns.duplicated()]  # Remove duplicate columns
        return result

    def compute_word_count_change_features(self, df, gap_list):
        """Computes word count change features for specified gaps per 'id'."""
        result = pd.DataFrame()  # Create an empty DataFrame to store the results
        for gap in gap_list:
            col_shift = f'word_count_shift{gap}'
            df[col_shift] = df.groupby('id')['word_count'].shift(gap)
            df[f'word_count_change{gap}'] = df['word_count'] - df[col_shift]
            df[f'word_count_abs_change{gap}'] = abs(df[f'word_count_change{gap}'])
            # Aggregate the results per 'id'
            id_features = df.groupby('id').agg({
                f'word_count_change{gap}': 'mean',  # You can choose different aggregation functions as needed
                f'word_count_abs_change{gap}': 'mean'
            }).reset_index()
            result = pd.concat([result, id_features], axis=1)  # Concatenate the results

        result = result.loc[:, ~result.columns.duplicated()]  # Remove duplicate columns
        return result

    # def compute_ratio_based_features(self, df):
    #     """Computes various ratio-based features per 'id'."""
    #     result = df.groupby('id').agg({
    #         'word_time_ratio': 'mean',  # You can choose different aggregation functions as needed
    #         'word_event_ratio': 'mean',
    #         'event_time_ratio': 'mean'
    #     }).reset_index()
    #     return result

    # ('word_count', ['nunique', 'max', 'quantile', 'sem', 'mean'])] It needs word_count_max to work
    
    def get_keyboard_mouse_feats(self, train_logs_df):

        # Creates two shift variables which lag the original variable by 1 and 2 periods respectively.
        event_df = train_logs_df[['id', 'event_id', 'down_event']].copy(deep=True)

        event_df['down_event_shift_1'] = event_df['down_event'].shift(periods=1)

        event_df = event_df[['id', 'event_id', 'down_event_shift_1', 'down_event']]

        ctrl_x_df = ((event_df['down_event_shift_1'] == 'Control') & (event_df['down_event'].str.lower() == 'x')).groupby(event_df['id']).sum().reset_index(name='count')

        # Creating a DataFrame that contains all counts at an id level.
        kb_shortcut_df = pd.DataFrame(event_df['id'].unique(), columns=['id'])
        kb_shortcut_df['ctrl_x_cnt'] = ctrl_x_df['count']

        # Calculating the proportion of mouse click events
        mouse_event_df = pd.DataFrame(train_logs_df['id'].unique(), columns=['id'])
        
        mouse_event_df['mouse_event_cnt'] = train_logs_df.groupby(train_logs_df['id'])['down_event'].apply(lambda x: (x.isin(['Leftclick', 'Rightclick', 'Middleclick', 'Unknownclick']).sum())).reset_index()['down_event']

        mouse_event_df['all_event_cnt'] = train_logs_df.groupby(train_logs_df['id'])['event_id'].max().reset_index()['event_id']

        mouse_event_df['mouse_event_perc'] = (mouse_event_df['mouse_event_cnt']/mouse_event_df['all_event_cnt'])*100.0
        
        return kb_shortcut_df.merge(mouse_event_df, on='id')

In [ ]:
import polars as pl

In [ ]:
activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',', 'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified']
text_changes = ['q', ' ', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':']


def count_by_values(df, colname, values):
    fts = df.select(pl.col('id').unique(maintain_order=True))
    for i, value in enumerate(values):
        tmp_df = df.group_by('id').agg(pl.col(colname).is_in([value]).sum().alias(f'{colname}_{i}_cnt'))
        fts  = fts.join(tmp_df, on='id', how='left') 
    return fts


def dev_feats(df):
    
    print("< Count by values features >")
    
    feats = count_by_values(df, 'activity', activities)
    feats = feats.join(count_by_values(df, 'text_change', text_changes), on='id', how='left') 
    feats = feats.join(count_by_values(df, 'down_event', events), on='id', how='left') 
    feats = feats.join(count_by_values(df, 'up_event', events), on='id', how='left') 

    return feats

In [ ]:
train_logs = pl.scan_csv(f'{data_dir}/train_logs.csv')
feat_ = dev_feats(train_logs).collect().to_pandas()

< Count by values features >


In [ ]:
feat_

,id,activity_0_cnt,activity_1_cnt,activity_2_cnt,activity_3_cnt,activity_4_cnt,text_change_0_cnt,text_change_1_cnt,text_change_2_cnt,text_change_3_cnt,...,up_event_6_cnt,up_event_7_cnt,up_event_8_cnt,up_event_9_cnt,up_event_10_cnt,up_event_11_cnt,up_event_12_cnt,up_event_13_cnt,up_event_14_cnt,up_event_15_cnt
0,001519c8,2010,417,120,7,0,1940,436,28,14,...,2,21,12,0,0,4,0,3,0,0
1,0022f953,1938,260,254,1,1,1698,432,18,24,...,49,15,21,3,2,6,0,3,0,0
2,0042269b,3515,439,175,7,0,3257,615,23,26,...,0,21,23,0,0,17,0,0,0,0
3,0059420b,1304,151,99,1,1,1146,281,13,3,...,0,13,3,0,0,3,2,2,0,0
4,0075873a,1942,517,72,0,0,1964,397,32,25,...,0,23,24,0,0,10,0,17,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,ffb8c745,3588,960,189,2,0,3605,813,59,42,...,0,43,32,0,0,7,0,7,0,0
2467,ffbef7e5,2395,60,148,1,0,1920,457,33,24,...,0,31,24,0,0,12,0,8,0,0
2468,ffccd6fd,2849,88,126,0,0,1031,1879,6,3,...,53,5,2,29,0,12,6,0,0,0
2469,ffec5b38,2895,276,71,0,0,2593,490,34,29,...,0,31,27,0,0,6,2,4,0,0


## Prepro

In [ ]:
# Split into sentences and paragraphs and compute aggregations
sentences = split_and_compute_lengths(train_essays, 'essay', r'[.?!]', 'sent')
sentence_aggregations = compute_aggregations(sentences, 'id', ['sent_len', 'sent_word_count'])

paragraphs = split_and_compute_lengths(train_essays, 'essay', '\n', 'paragraph')
paragraph_aggregations = compute_aggregations(paragraphs, 'id', ['paragraph_len', 'paragraph_word_count'])

paragraph_ratios = calculate_relative_paragraph_sizes(train_essays, 'essay')

In [ ]:
# Instantiate your classes
feature_engineering = FeatureEngineering()
feature_stats = FeatureStats()
preprocessor = Preprocessor()

In [ ]:
# Compute basic features
input_words_features = feature_engineering.get_input_words(train_logs_df)
word_counts = feature_stats.get_word_counts(train_logs_df, 'id', 'word_count')

# Additional features from Preprocessor
activity_counts = preprocessor.activity_counts(train_logs_df).reset_index()
event_counts = preprocessor.event_counts(train_logs_df, 'down_event').reset_index()
text_change_counts = preprocessor.text_change_counts(train_logs_df).reset_index()
punctuation_counts = preprocessor.match_punctuations(train_logs_df).reset_index()

c:\Users\luis.pinto1\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\luis.pinto1\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
# Define the gap list for time gap features
gap_list = [1, 2, 3, 5, 10, 20, 50, 100]

# Compute time gap features
time_gap_features = preprocessor.compute_time_gaps(train_logs_df, gap_list)
time_gap_features.columns = ['{}_{}'.format(col1, col2) for col1, col2 in time_gap_features.columns]
time_gap_features = time_gap_features.reset_index()

# Compute pause-related features
pause_features = preprocessor.aggregate_pause_features(train_logs_df).reset_index()

# Compute process variance
process_variance = preprocessor.aggregate_process_variance(train_logs_df).reset_index()

# Create additional time features
time_features = preprocessor.create_time_features(train_logs_df)
additional_time_features = preprocessor.create_additional_time_features(train_logs_df)

# Text IDF features
text_features = preprocessor.make_text_features(train_logs_df)

c:\Users\luis.pinto1\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
cursor_feats = preprocessor.compute_cursor_position_change_features(train_logs_df, gap_list)
word_counts_feats = preprocessor.compute_word_count_change_features(train_logs_df, gap_list)
# ratios_feats = preprocessor.compute_ratio_based_features(train_logs_df)

In [ ]:
mouse_keyboard_feats = preprocessor.get_keyboard_mouse_feats(train_logs_df)

## Combine all features

In [ ]:
list_of_dfs = [sentence_aggregations, paragraph_aggregations, paragraph_ratios, input_words_features,
            word_counts, activity_counts, event_counts, text_change_counts, punctuation_counts,
            time_gap_features, pause_features, process_variance, time_features, additional_time_features,
            text_features, cursor_feats, word_counts_feats, mouse_keyboard_feats]

# Define a function to merge two DataFrames on 'id' with 'inner' join
merge_two_dfs = lambda left, right: pd.merge(left, right, on='id', how='inner')

# Use reduce to iteratively merge all DataFrames in the list
merged_df = reduce(merge_two_dfs, list_of_dfs)

In [ ]:
punctuation_counts

down_event,id,!,&,',(,+,",",-,.,/,...,,¡,¿,Â´,Ä±,Å,Ë,â,ä,Total
0,001519c8,0,0,3,0,0,12,0,21,0,...,0,0,0,0,0,0,0,0,0,2556
1,0022f953,0,0,3,0,0,21,5,15,0,...,0,0,0,0,0,0,0,0,0,2445
2,0042269b,0,0,0,0,0,23,1,21,0,...,0,0,0,0,0,0,0,0,0,4134
3,0059420b,0,0,2,0,0,3,0,13,0,...,0,0,0,0,0,0,0,0,0,1556
4,0075873a,0,0,17,0,0,24,0,23,0,...,0,0,0,0,0,0,0,0,0,2529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,ffb8c745,0,0,7,0,0,32,1,43,0,...,0,0,0,0,0,0,0,0,0,4735
2467,ffbef7e5,0,0,8,0,0,24,0,31,0,...,0,0,0,0,0,0,0,0,0,2604
2468,ffccd6fd,0,0,0,0,0,2,0,5,0,...,0,0,0,0,0,0,0,0,0,3063
2469,ffec5b38,0,0,4,0,0,27,1,31,0,...,0,0,0,0,0,0,0,0,0,3235


In [ ]:
for df_ in list_of_dfs:
    for col in df_.columns:
        if 'ArrowUp' in col:
            print(df_)

down_event        id  down_event_ArrowRight  down_event_ArrowLeft  \
0           001519c8                    0.0                   0.0   
1           0022f953                    0.0                   0.0   
2           0042269b                    0.0                   NaN   
3           0059420b                    NaN                   NaN   
4           0075873a                    NaN                   NaN   
...              ...                    ...                   ...   
2466        ffb8c745                    NaN                   NaN   
2467        ffbef7e5                    NaN                   NaN   
2468        ffccd6fd                    0.0                   0.0   
2469        ffec5b38                    NaN                   NaN   
2470        fff05981                    0.0                   0.0   

down_event  down_event_ArrowDown  down_event_ArrowUp  down_event_CapsLock  \
0                            NaN                 NaN                  NaN   
1                

In [ ]:
input_words_features

,id,input_word_count,input_word_length_mean,input_word_length_max,input_word_length_std
0,001519c8,366.0,5.325137,20.0,3.487804
1,0022f953,385.0,4.410390,33.0,3.199496
2,0042269b,627.0,5.446571,25.0,3.474895
3,0059420b,251.0,4.609562,19.0,2.949601
4,0075873a,412.0,4.766990,18.0,2.986064
...,...,...,...,...,...
2466,ffb8c745,734.0,4.915531,20.0,3.001989
2467,ffbef7e5,470.0,4.085106,13.0,2.231589
2468,ffccd6fd,222.0,4.644144,15.0,2.707087
2469,ffec5b38,500.0,5.294000,24.0,3.541689


In [ ]:
merged_df.merge(train_scores_df, on='id').to_csv('example_features.csv', index=False)

In [ ]:
merged_df

,id,sent_len_count,sent_len_mean,sent_len_median,sent_len_std,sent_len_min,sent_len_max,sent_len_var,sent_len_sem,sent_word_count_count,...,word_count_abs_change50,word_count_change100,word_count_abs_change100,ctrl_bksp_cnt,ctrl_c_cnt,ctrl_v_cnt,ctrl_x_cnt,mouse_event_cnt,all_event_cnt,mouse_event_perc
0,001519c8,14,106.142857,119.5,41.128050,31,196,1691.516484,10.991934,14,...,5.064619,10.060643,10.060643,0,0,0,0,92,2557,3.597966
1,0022f953,15,107.666667,92.0,64.713287,19,226,4187.809524,16.708899,15,...,6.634359,13.378505,13.397196,0,0,0,0,56,2454,2.281989
2,0042269b,19,133.842105,139.0,33.480115,73,189,1120.918129,7.680865,19,...,6.161527,9.679386,10.833499,0,0,0,0,130,4136,3.143133
3,0059420b,13,86.846154,80.0,33.195999,39,144,1101.974359,9.206914,13,...,6.879150,13.580357,13.580357,0,1,2,0,19,1556,1.221080
4,0075873a,16,86.812500,74.0,44.094170,22,182,1944.295833,11.023543,16,...,5.958484,9.937063,11.323324,0,0,0,0,33,2531,1.303832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,ffb8c745,13,121.076923,132.0,40.376275,55,180,1630.243590,11.198364,13,...,8.452975,5.897392,16.040526,0,0,0,0,24,4739,0.506436
2467,ffbef7e5,29,78.310345,67.0,40.481127,20,175,1638.721675,7.517157,29,...,8.516445,16.759585,16.759585,0,0,0,0,38,2604,1.459293
2468,ffccd6fd,4,277.000000,274.5,77.395090,200,359,5990.000000,38.697545,4,...,3.269831,6.522781,6.522781,0,0,0,0,9,3063,0.293830
2469,ffec5b38,27,92.592593,98.0,33.747090,36,176,1138.866097,6.494631,27,...,6.435777,12.716104,12.716741,0,0,0,0,14,3242,0.431832


In [ ]:
import pandas as pd
import numpy as np

def filter_columns(df):
    # Initialize lists to store column names for each condition
    all_nan_columns = []
    all_inf_columns = []
    single_value_columns = []

    # Iterate through columns
    for col in df.columns:
        # Check if the column contains numeric data
        if pd.api.types.is_numeric_dtype(df[col]):
            # Check for all NaN values
            if df[col].isna().all():
                all_nan_columns.append(col)
            else:
                # Check for all infinite values (positive or negative infinity)
                if np.isinf(df[col]).all():
                    all_inf_columns.append(col)
                else:
                    # Check for more than 90% of columns with only one unique value
                    unique_value_count = df[col].nunique()
                    total_count = df.shape[0]
                    if unique_value_count == 1 and (total_count - df[col].isna().sum()) / total_count > 0.8:
                        single_value_columns.append(col)

    return {
        "AllNaNColumns": all_nan_columns,
        "AllInfColumns": all_inf_columns,
        "SingleValueColumns": single_value_columns
    }

# Example usage:
# Assuming 'df' is your DataFrame
filtered_columns = filter_columns(merged_df)
print("Columns with all NaNs:", filtered_columns["AllNaNColumns"])
print("Columns with all infinite values:", filtered_columns["AllInfColumns"])
print("Columns with more than 90% of the same value:", filtered_columns["SingleValueColumns"])

Columns with all NaNs: []
Columns with all infinite values: []
Columns with more than 90% of the same value: []
